In [1]:
!!pip install litellm

['Collecting litellm',
 '  Downloading litellm-1.74.8-py3-none-any.whl.metadata (40 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/40.4 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b40.4/40.4 kB\x1b \x1b1.4 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hRequirement already satisfied: aiohttp>=3.10 in /usr/local/lib/python3.11/dist-packages (from litellm) (3.12.14)',
 'Requirement already satisfied: click in /usr/local/lib/python3.11/dist-packages (from litellm) (8.2.1)',
 'Requirement already satisfied: httpx>=0.23.0 in /usr/local/lib/python3.11/dist-packages (from litellm) (0.28.1)',
 'Requirement already satisfied: importlib-metadata>=6.8.0 in /usr/local/lib/python3.11/dist-packages (from litellm) (8.7.0)',
 'Requirement already satisfied: jinja2<4.0.0,>=3.1.2 in /usr/local/lib/python3.11/dist-packages (from litellm) (3.1.6)',
 'Requirement already satisfied: jsonschema<5.0.0,>=4.22.0 in /usr/local

In [2]:
import os
import json
import time
import traceback
import inspect
from typing import List, Dict, Any
from dataclasses import dataclass, field

try:
    from litellm import completion
except ImportError:
    print("Please install LiteLLM: pip install litellm")
    raise

#---------- CONFIG ----------------------------
GROQ_MODEL = "meta-llama/llama-4-scout-17b-16e-instruct"
GROQ_KEY = os.environ.get("GROQ_API_KEY")
if not GROQ_KEY:
    import getpass
    GROQ_KEY = getpass.getpass("Enter your Groq API key:")


#---------- TOOL REGISTRATION DECORATOR ------------------
tools = {}
tools_by_tag = {}

def register_tool(tool_name=None, description=None, parameters_override=None, terminal=False, tags=None):
    def decorator(func):
        signature = inspect.signature(func)
        type_hints = getattr(func, '__annotations__', {})
        args_schema = {
            "type": "object",
            "properties": {},
            "required": []
        }
        for param_name, param in signature.parameters.items():
            if param_name in ["action_context", "action_agent"]: continue
            param_type = type_hints.get(param_name, str)
            if param_type == str: json_type = "string"
            elif param_type == int: json_type = "integer"
            elif param_type == float: json_type = "number"
            elif param_type == bool: json_type = "boolean"
            elif param_type == dict: json_type = "object"
            elif param_type == list: json_type = "array"
            else: json_type = "string"
            args_schema["properties"][param_name] = {"type": json_type}
            if param.default == inspect.Parameter.empty:
                args_schema["required"].append(param_name)
        if parameters_override:
            args_schema = parameters_override
        tools[func.__name__] = {
            "description": description or func.__doc__ or "",
            "parameters": args_schema,
            "function": func,
            "terminal": terminal,
            "tags": tags or []
        }
        for tag in (tags or []):
            if tag not in tools_by_tag: tools_by_tag[tag] = []
            tools_by_tag[tag].append(func.__name__)
        return func
    return decorator

#------------- AGENT PRIMITIVES -----------------
class Agent:
    def __init__(self, name: str):
        self.name = name
    def run(self, input_data: Any) -> Any:
        raise NotImplementedError

@dataclass
class Memory:
    items: list = field(default_factory=list)
    def add(self, entry: dict): self.items.append(entry)
    def get(self): return self.items

#------------- LITELLM + GROQ TOOLCALLER ----------------
def prompt_expert(action_context, description_of_expert, prompt) -> Any:
    print(f"\n[LLM Expert: {description_of_expert}]\nPrompt (truncated): {prompt[:290]}...\n")
    try:
        resp = completion(
            model=f"groq/{GROQ_MODEL}",
            messages=[
                {"role": "system", "content": description_of_expert},
                {"role": "user", "content": prompt}
            ],
            api_key=GROQ_KEY,
            max_tokens=400
        )
        text = None
        if hasattr(resp, "choices") and hasattr(resp.choices[0], "message") and hasattr(resp.choices[0].message, "content"):
            text = resp.choices[0].message.content
        elif hasattr(resp, "choices") and hasattr(resp.choices[0], "text"):
            text = resp.choices[0].text
        elif isinstance(resp, dict) and "choices" in resp:
            if "message" in resp["choices"][0]:
                text = resp["choices"][0]["message"]["content"]
            elif "text" in resp["choices"][0]:
                text = resp["choices"][0]["text"]
        elif isinstance(resp, str):
            text = resp
        else:
            text = str(resp)
        try:
            return json.loads(text)
        except Exception:
            return text.strip() if isinstance(text, str) else text
    except Exception as e:
        print(f"Error in prompt_expert LLM call: {e}")
        return f"LLM error: {e}"

#---------- TOOL DEFINITIONS ----------------------------
@register_tool(tags=["invoice_processing", "categorization"])
def categorize_expenditure(action_context, description: str) -> str:
    categories = [
        "Office Supplies", "IT Equipment", "Software Licenses", "Consulting Services",
        "Travel Expenses", "Marketing", "Training & Development", "Facilities Maintenance",
        "Utilities", "Legal Services", "Insurance", "Medical Services", "Payroll",
        "Research & Development", "Manufacturing Supplies", "Construction", "Logistics",
        "Customer Support", "Security Services", "Miscellaneous"
    ]
    return prompt_expert(
        action_context=action_context,
        description_of_expert="A senior financial analyst with deep expertise in corporate spending categorization.",
        prompt=f"Given the following description: '{description}', classify the expense into one of these categories:\n{categories}\nRespond only with the best category."
    )

@register_tool(tags=["invoice_processing", "validation"])
def check_purchasing_rules(action_context, invoice_data: dict, rules_path="config/purchasing_rules.txt") -> dict:
    try:
        with open(rules_path, "r") as f:
            purchasing_rules = f.read()
    except FileNotFoundError:
        return {"compliant": True, "issues": f"No purchasing rules file found at {rules_path}. All invoices assumed compliant."}
    prompt = f"""
Given this invoice data:
{json.dumps(invoice_data, indent=2)}

and these company purchasing rules:
{purchasing_rules}

Analyze the invoice and respond in JSON:
{{"compliant": true|false, "issues": "<brief string>"}}
"""
    return prompt_expert(
        action_context=action_context,
        description_of_expert="A corporate procurement compliance officer with extensive knowledge of purchasing policies.",
        prompt=prompt
    )

#---------- SUBAGENTS ----------------------------

class InvoiceExtractionAgent(Agent):
    """Extracts structured invoice data (can swap for LLM/NLP)"""
    def run(self, invoice_text: str) -> Dict:
        # You can upgrade to LLM-based extraction
        # For now, mimic field extraction
        ret = {
            "invoice_id": "INV-2025-0042",
            "vendor": "AcmeIT",
            "description": "Laptop computers for engineering team",
            "amount": 7700,
            "date": "2025-07-17",
            "line_items": [
                {"item": "Laptop", "qty": 5, "unit_price": 1500}
            ]
        }
        if "Acme" in invoice_text: ret["vendor"] = "AcmeIT"
        if "Training" in invoice_text:
            ret["description"] = "Online training course for staff"
            ret["amount"] = 450
        return ret

class InvoiceCategorizationAgent(Agent):
    """Calls the categorization expert tool."""
    def run(self, invoice_data: Dict) -> str:
        return categorize_expenditure(None, description=invoice_data["description"])

class InvoiceValidationAgent(Agent):
    """Calls the purchase policy expert tool."""
    def __init__(self, rules_path="config/purchasing_rules.txt"):
        self.rules_path = rules_path
    def run(self, invoice_data: Dict) -> Dict:
        return check_purchasing_rules(None, invoice_data=invoice_data, rules_path=self.rules_path)

#---------- COORDINATOR/PIPELINE AGENT ----------------------------
class InvoiceProcessingCoordinator(Agent):
    def __init__(self, rules_path="config/purchasing_rules.txt"):
        super().__init__("Coordinator")
        self.extractor = InvoiceExtractionAgent("Extractor")
        self.categorizer = InvoiceCategorizationAgent("Categorizer")
        self.validator = InvoiceValidationAgent(rules_path)
        self.memory = Memory()

    def run(self, invoice_text: str) -> Dict:
        # 1. Extract fields
        invoice_data = self.extractor.run(invoice_text)
        self.memory.add({"stage": "extraction", "data": invoice_data})
        # 2. Categorize
        category = self.categorizer.run(invoice_data)
        invoice_data["category"] = category
        self.memory.add({"stage": "categorization", "category": category})
        # 3. Validate
        compliance = self.validator.run(invoice_data)
        invoice_data["policy_compliance"] = compliance
        self.memory.add({"stage": "compliance", "result": compliance})
        return invoice_data

#--------- MAIN USAGE ---------
if __name__ == "__main__":
    invoice_text = """
    Vendor: AcmeIT
    For: Laptop computers for engineering team
    Amount: $7700
    Date: 2025-07-17
    """
    agent = InvoiceProcessingCoordinator(rules_path="purchasing_rules.txt")
    result = agent.run(invoice_text)
    print("\nFINAL STRUCTURED OUTPUT:")
    print(json.dumps(result, indent=2))


Enter your Groq API key:··········

[LLM Expert: A senior financial analyst with deep expertise in corporate spending categorization.]
Prompt (truncated): Given the following description: 'Laptop computers for engineering team', classify the expense into one of these categories:
['Office Supplies', 'IT Equipment', 'Software Licenses', 'Consulting Services', 'Travel Expenses', 'Marketing', 'Training & Development', 'Facilities Maintenance', '...


[LLM Expert: A corporate procurement compliance officer with extensive knowledge of purchasing policies.]
Prompt (truncated): 
Given this invoice data:
{
  "invoice_id": "INV-2025-0042",
  "vendor": "AcmeIT",
  "description": "Laptop computers for engineering team",
  "amount": 7700,
  "date": "2025-07-17",
  "line_items": [
    {
      "item": "Laptop",
      "qty": 5,
      "unit_price": 1500
    }
  ],
  "cate...


FINAL STRUCTURED OUTPUT:
{
  "invoice_id": "INV-2025-0042",
  "vendor": "AcmeIT",
  "description": "Laptop computers for engine